Let's first import some modules and load .env

In [73]:
import pandas as pd
import psycopg2
import dotenv
import os
import matplotlib.pyplot as plt
import numpy as np

dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD", "_KjE6\\2_JX0UGTJdZYl")
PG_USER = os.getenv("PG_USER", "busit_87")
PG_HOST = os.getenv("PG_HOST", "fuji.ucll.be")
PG_PORT = os.getenv("PG_PORT", "52425")
PG_DATABASE = os.getenv("PG_DB", "international_week")

and connect to the database

In [74]:
conn = psycopg2.connect(
    host="fuji.ucll.be",
    database="international_week",
    user=PG_USER,
    password=PG_PASSWORD,
    port=52425,
    sslmode="require",
)


Here we define a function that would cound the amount of actions. As parameters it has the game_id, pov_team (a certain team in that game) and a filter_sql which is a filter that we add at the end of a long query which shows us the SPADL table with some column that we add by using CTEs.

In [75]:

def count_actions(game_id, pov_team, conn, filter_sql):
    query = f"""
WITH action_changes AS (
        SELECT
            a.*,
            LAG(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS prev_team_id,
            LEAD(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS next_team_id
        FROM
            public.spadl_actions a
        WHERE
            a.game_id = '{game_id}'
    ),
    possession_markers AS (
        SELECT
            *,
            CASE WHEN prev_team_id IS NULL OR team_id != prev_team_id THEN 1 ELSE 0 END AS is_new_possession
        FROM
            action_changes
    ),
    possession_groups AS (
        SELECT
            *,
            SUM(is_new_possession) OVER (ORDER BY period_id, seconds, id) AS possession_group
        FROM
            possession_markers
    ),
    count_rows_in_possession_group AS (
        SELECT
            *,
            COUNT(*) OVER (PARTITION BY possession_group ORDER BY period_id, seconds, id) AS rows_in_possession_group
        FROM
            possession_groups
    ),
    initial_value_per_group AS (
        SELECT
            possession_group,
            MIN(seconds) AS initial_seconds
        FROM
            count_rows_in_possession_group
        GROUP BY
            possession_group
    ),
    possession_stats AS (
        SELECT
            possession_group,
            team_id,
            COUNT(*) AS action_count,
            MAX(id) AS last_action_id
        FROM
            count_rows_in_possession_group
        GROUP BY
            possession_group, team_id
    ),
    seconds_difference AS (
        SELECT
            c.*, 
            (c.seconds - i.initial_seconds) AS seconds_difference,
            ABS(c.start_x - c.end_x) AS action_distance
        FROM
            count_rows_in_possession_group c
        JOIN 
            initial_value_per_group i
        ON 
            c.possession_group = i.possession_group
    ),
    filters AS (
        SELECT *
        FROM seconds_difference
        WHERE team_id = '{pov_team}'
        AND possession_group IN (
            SELECT possession_group FROM possession_stats WHERE action_count >= 3
        )
    ),
    start_with_ball_table AS (
        SELECT *,
            CASE 
                WHEN FIRST_VALUE(prev_team_id) OVER (ORDER BY id) IS NULL AND FIRST_VALUE(end_x) OVER (ORDER BY id) < 52.5
                THEN 'LEFT_SIDE' 
                WHEN FIRST_VALUE(prev_team_id) OVER (ORDER BY id) IS NOT NULL AND FIRST_VALUE(end_x) OVER (ORDER BY id) < 52.5
                THEN 'RIGHT_SIDE'
                WHEN FIRST_VALUE(prev_team_id) OVER (ORDER BY id) IS NULL AND FIRST_VALUE(end_x) OVER (ORDER BY id) > 52.5
                THEN 'RIGHT_SIDE' 
                ELSE 'LEFT_SIDE' 
            END AS start_with_ball
        FROM filters
    ),
    start_with_period AS (
        SELECT *,
            CASE 
                WHEN start_with_ball = 'LEFT_SIDE' and period_id = 2
                THEN 'RIGHT_SIDE' 
                WHEN start_with_ball = 'RIGHT_SIDE' and period_id = 2
                THEN 'LEFT_SIDE' 
                WHEN start_with_ball = 'LEFT_SIDE' and period_id = 1
                THEN 'LEFT_SIDE' 
                WHEN start_with_ball = 'RIGHT_SIDE' and period_id = 1
                THEN 'RIGHT_SIDE' 
            END AS update_start_with_ball
        FROM start_with_ball_table
    ),
    direction AS (
        SELECT *,
            CASE 
            WHEN start_x > end_x AND update_start_with_ball = 'RIGHT_SIDE' THEN 'FORWARD'
            WHEN start_x < end_x AND update_start_with_ball = 'RIGHT_SIDE' THEN 'BACKWARD'
            WHEN start_x < end_x AND update_start_with_ball = 'LEFT_SIDE' THEN 'FORWARD'
            WHEN start_x > end_x AND update_start_with_ball = 'LEFT_SIDE' THEN 'BACKWARD'
            ELSE NULL
        END AS ballMoveDirection
        FROM start_with_period
    ),
	ballSidePositionAbsolute AS (
		select *,
		case
		when start_x >= 0 and start_x < 35 then 'LEFT_SIDE'
		when start_x >= 35 and start_x < 70 then 'MIDDLE'
		when start_x >= 70 and start_x < 105 then 'RIGHT_SIDE'
		else null
		end as actionStartSideAbsolute
		
		from direction
	),

	    ballSide AS (
        SELECT *,
            CASE 
                WHEN actionStartSideAbsolute = 'RIGHT_SIDE' AND update_start_with_ball = 'LEFT_SIDE' THEN '3'
                WHEN actionStartSideAbsolute = 'LEFT_SIDE' AND update_start_with_ball = 'RIGHT_SIDE' THEN '3'
                WHEN actionStartSideAbsolute = 'MIDDLE' THEN '2'
                WHEN actionStartSideAbsolute = 'RIGHT_SIDE' AND update_start_with_ball = 'RIGHT_SIDE' THEN '1'
                WHEN actionStartSideAbsolute = 'LEFT_SIDE' AND update_start_with_ball = 'LEFT_SIDE' THEN '1'
                ELSE NULL
            END AS possition_ball_by_action 
        FROM ballSidePositionAbsolute
    )
select count(*) from ballSide  {filter_sql}

    """
    return pd.read_sql_query(query, conn)



This function takes in parameter a small query that will be added to the long one previously showed. It will then count all the actions of both teams within a match and this for every matches played. It will then return the average of that action match for all teams in every match played.

In [76]:
def count_average_actions_per_team(query):

    query_match = """
    SELECT 
        m.match_id, 
        m.home_team_id, 
        ht.team_name AS home_team_name, 
        m.away_team_id, 
        at.team_name AS away_team_name
    FROM public.matches m
    JOIN public.teams ht ON m.home_team_id = ht.team_id
    JOIN public.teams at ON m.away_team_id = at.team_id
    ORDER BY m.match_id ASC;
    """

    df = pd.read_sql_query(query_match, conn)

    team_stats = {}

    for row in df.itertuples(index=False):
        match_id, first_team_id, second_team_id, first_team_name, second_team_name = row.match_id, row.home_team_id, row.away_team_id, row.home_team_name, row.away_team_name

        first_actions = count_actions(match_id, first_team_id, conn, query) #we can reverse those lines if we want to have the for example all passes that the opposing team does
        second_actions = count_actions(match_id, second_team_id, conn, query) #but it's more for defencing actions.


        if first_team_name not in team_stats:
            team_stats[first_team_name] = {'total_actions': 0, 'match_count': 0}
        team_stats[first_team_name]['total_actions'] += first_actions
        team_stats[first_team_name]['match_count'] += 1

        if second_team_name not in team_stats:
            team_stats[second_team_name] = {'total_actions': 0, 'match_count': 0}
        team_stats[second_team_name]['total_actions'] += second_actions
        team_stats[second_team_name]['match_count'] += 1

    team_data = []
    for team_name, stats in team_stats.items():
        avg_actions = stats['total_actions'] / stats['match_count']
        team_data.append((team_name, avg_actions))

    average_df = pd.DataFrame(team_data, columns=['team_name', 'avg_actions'])

    average_df_copy = average_df.copy()

    average_df_copy['avg_actions'] = (
        average_df_copy['avg_actions']
        .astype(str)
        .str.replace(r'count\s*0\s*', '', regex=True)
        .str.strip()
        .replace('', '0')  
        .astype(float)  
    )

    return average_df_copy

So here we will define a small query to filter all actions, that happen within 10 seconds after a team gain a possession which is of type pass and which of direction forward. Then we pass it to the function count_average_actions_per_team and then we can display a dataframe with the average of those passes for every teams in the league of all their matches

In [77]:
filter_sql_forward = """WHERE seconds_difference <= 10 AND action_type IN ('0') AND ballMoveDirection = 'FORWARD'"""
df_filter_sql_forward = count_average_actions_per_team(filter_sql_forward)
df_filter_sql_forward

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1515071174.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_match, conn)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1726669913.py:139: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


,team_name,avg_actions
0,Club Brugge,124.947368
1,Mechelen,105.277778
2,Dender,98.000000
3,Union Saint-Gilloise,107.333333
4,Beerschot,94.176471
5,OH Leuven,90.611111
6,Anderlecht,100.000000
7,Sint-Truiden,105.944444
8,Genk,110.421053
9,Standard Liège,80.578947


Let's do it now for all forward passes which were made within 10 seconds after getting the ball, that are longer that, 5m, 10m and 20m

In [78]:
filter_sql_forward_and_5m_distance = """WHERE seconds_difference <= 10 AND action_type IN ('0') AND ballMoveDirection = 'FORWARD' AND action_distance>5"""
df_filter_sql_forward_and_5m_distance = count_average_actions_per_team(filter_sql_forward_and_5m_distance)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1515071174.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_match, conn)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1726669913.py:139: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [79]:
filter_sql_forward_and_10m_distance = """WHERE seconds_difference <= 10 AND action_type IN ('0') AND ballMoveDirection = 'FORWARD' AND action_distance>10"""
df_filter_sql_forward_and_10m_distance = count_average_actions_per_team(filter_sql_forward_and_10m_distance)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1515071174.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_match, conn)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1726669913.py:139: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [ ]:
filter_sql_forward_and_20m_distance = """WHERE seconds_difference <= 10 AND action_type IN ('0') AND ballMoveDirection = 'FORWARD' AND action_distance>20"""
df_filter_sql_forward_and_20m_distance = count_average_actions_per_team(filter_sql_forward_and_20m_distance)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1515071174.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_match, conn)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_16028\1726669913.py:139: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [ ]:
df_merged = df_filter_sql_forward.merge(
    df_filter_sql_forward_and_5m_distance, on='team_name', suffixes=('_passes_forward', '_passes_forward_5m')
).merge(
    df_filter_sql_forward_and_10m_distance, on='team_name', suffixes=('', '_passes_forward_10m')
).merge(
    df_filter_sql_forward_and_20m_distance, on='team_name', suffixes=('', '_passes_forward_20m')
)

df_merged = df_merged.rename(columns={'avg_actions': 'avg_actions_passes_forward_10m'})

In [ ]:
df_merged

We can also add columns that would show ratios between 5m, 10m 20m forward passes and all forward passes

In [ ]:
df_merged['ratio 5m'] = df_merged['avg_actions_passes_forward_5m']/df_merged['avg_actions_passes_forward']
df_merged['ratio 10m'] = df_merged['avg_actions_passes_forward_10m']/df_merged['avg_actions_passes_forward']
df_merged['ratio 20m'] = df_merged['avg_actions_passes_forward_20m']/df_merged['avg_actions_passes_forward']
df_merged[['team_name','ratio 5m', 'ratio 10m', 'ratio 20m']]

And now let's display all those ratios in a graph

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(df_merged['team_name'], df_merged['ratio 5m'], label='5m Ratio', alpha=0.6)
plt.bar(df_merged['team_name'], df_merged['ratio 10m'], label='10m Ratio', alpha=0.6)
plt.bar(df_merged['team_name'], df_merged['ratio 20m'], label='20m Ratio', alpha=0.6)
plt.xlabel('Team Name')
plt.ylabel('Ratio')
plt.title('Passes Forward Ratios for Teams')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()

We can also use the column possition_ball_by_action that shows where is does the ball start where 1 is the closest to the goalkeeper of a team to show where is the starting point of the passes that are longer than 5m and that goes forward and that are made within 10 seconds after a teams gain the possession

In [ ]:

filter_sql_forward_and_5m_distance_on_last_side="""WHERE seconds_difference <= 10 AND action_type IN ('0') AND ballMoveDirection = 'FORWARD' AND action_distance>5 AND possition_ball_by_action = '3'"""
df_filter_sql_forward_and_5m_distance_on_last_side = count_average_actions_per_team(filter_sql_forward_and_5m_distance_on_last_side)

In [ ]:
filter_sql_forward_and_5m_distance_on_middel_side="""WHERE seconds_difference <= 10 AND action_type IN ('0') AND ballMoveDirection = 'FORWARD' AND action_distance>5 AND possition_ball_by_action = '2'"""
df_filter_sql_forward_and_5m_distance_on_middel_side = count_average_actions_per_team(filter_sql_forward_and_5m_distance_on_middel_side)

In [ ]:
filter_sql_forward_and_5m_distance_on_first_side="""WHERE seconds_difference <= 10 AND action_type IN ('0') AND ballMoveDirection = 'FORWARD' AND action_distance>5 AND possition_ball_by_action = '1'"""
df_filter_sql_forward_and_5m_distance_on_first_side = count_average_actions_per_team(filter_sql_forward_and_5m_distance_on_first_side)

In [ ]:
df_merged_side = df_filter_sql_forward_and_5m_distance.merge(
    df_filter_sql_forward_and_5m_distance_on_first_side, on='team_name', suffixes=('_passes_forward', '_passes_forward_5m_first_side')
).merge(
    df_filter_sql_forward_and_5m_distance_on_middel_side, on='team_name', suffixes=('', '_passes_forward_5m_middel_side')
).merge(
    df_filter_sql_forward_and_5m_distance_on_last_side, on='team_name', suffixes=('', '_passes_forward_5m_last_side')
)
df_merged_side = df_merged.rename(columns={'avg_actions': 'avg_actions_passes_forward_5m_middel_side'})

In [ ]:
df_merged_side = df_filter_sql_forward_and_5m_distance.merge(
    df_filter_sql_forward_and_5m_distance_on_first_side, on='team_name', suffixes=('_passes_forward', '_passes_forward_5m_first_side')
).merge(
    df_filter_sql_forward_and_5m_distance_on_middel_side, on='team_name', suffixes=('', '_passes_forward_5m_middel_side')
).merge(
    df_filter_sql_forward_and_5m_distance_on_last_side, on='team_name', suffixes=('', '_passes_forward_5m_last_side')
)
df_merged_side = df_merged.rename(columns={'avg_actions': 'avg_actions_passes_forward_5m_middel_side'})
df_merged_side

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(df_merged['team_name'], df_merged['ratio 5m'], label='5m Ratio', alpha=0.6)
plt.bar(df_merged['team_name'], df_merged['ratio 10m'], label='10m Ratio', alpha=0.6)
plt.bar(df_merged['team_name'], df_merged['ratio 20m'], label='20m Ratio', alpha=0.6)
plt.xlabel('Team Name')
plt.ylabel('Ratio')
plt.title('Passes Forward Ratios for Teams')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()

The same way we did before, we can show only ratios of those passes comparing to all passes that are made forward

In [ ]:
df_merged_side['ratio first'] = df_merged_side['avg_actions_passes_forward_5m_first_side']/df_merged_side['avg_actions_passes_forward']
df_merged_side['ratio middel'] = df_merged_side['avg_actions_passes_forward_5m_middel_side']/df_merged_side['avg_actions_passes_forward']
df_merged_side['ratio last'] = df_merged_side['avg_actions_passes_forward_5m_last_side']/df_merged_side['avg_actions_passes_forward']
df_merged_side[['team_name','ratio first', 'ratio middel', 'ratio first']]

And plot it

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(df_merged['team_name'], df_merged['ratio first'], label='ratio first', alpha=0.6)
plt.bar(df_merged['team_name'], df_merged['ratio middel'], label='ratio middel', alpha=0.6)
plt.bar(df_merged['team_name'], df_merged['ratio last'], label='ratio last', alpha=0.6)
plt.xlabel('Team Name')
plt.ylabel('Ratio')
plt.title('5m Passes Forward Per sides for Teams')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()